# Using AstraPy with vector database

Work with AstraPy

Make sure you have a Database and get ready to supply the Token string
([Read about astra setup](https://docs.datastax.com/en/astra/home/astra.html) for details. Remember you need a **custom Token** with role [Database Administrator](https://awesome-astra.github.io/docs/pages/astra/create-token/)).

Example values:

- API Endpoint: https://01234567-89ab-cdef-0123-456789abcdef-us-east1.apps.astra.datastax.com
- Token: AstraCS:6gBhNmsk135.... (it must have a role of at least "Database Administrator")

### Set up your python environment




In [ ]:
%pip install astrapy --quiet

### Import needed libraries



In [ ]:
import os, json
import uuid
from getpass import getpass

from astrapy.db import AstraDB

ASTRA_DB_API_ENDPOINT = input("ASTRA_DB_API_ENDPOINT = ")
ASTRA_DB_APPLICATION_TOKEN = getpass("ASTRA_DB_APPLICATION_TOKEN = ")


### Astra Setup



### Astra Variables

In [ ]:
### Client setup
astra_db = AstraDB(
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

In [ ]:
# Create and view collections

In [ ]:
# Just in case it already exists, delete it
res = astra_db.delete_collection(collection_name="test_collection")
print(res)

# Now create the collection afresh
res = astra_db.create_collection(collection_name="test_collection", dimension=5)
collection = astra_db.collection(collection_name="test_collection")
print(res)

res = astra_db.get_collections()
print(res)

# Insert one document

When working with vector stores, there are reserved fields `$vector`` and `$similarity`.  The `$` is how these reserved words are used in the system.

In [ ]:
# Insert into vector collection
json_query = {
        "_id": "4",
        "name": "Coded Cleats Copy",
        "description": "ChatGPT integrated sneakers that talk to you",
        "$vector": [0.25, 0.25, 0.25, 0.25, 0.25],
    }

res = collection.insert_one(document=json_query)
print(res)

In [ ]:
# Insert non-vector document

json_query = {
        "_id": "Cliff1",
        "first_name": "Cliff",
        "last_name": "Wicklow",
    }
response = collection.insert_one(document=json_query)
print(response)


# Insert multiple documents

In [ ]:
json_query = [
        {
            "_id": "1",
            "name": "Coded Cleats",
            "description": "ChatGPT integrated sneakers that talk to you",
            "$vector": [0.1, 0.15, 0.3, 0.12, 0.05],
        },
        {
            "_id": "2",
            "name": "Logic Layers",
            "description": "An AI quilt to help you sleep forever",
            "$vector": [0.45, 0.09, 0.01, 0.2, 0.11],
        },
        {
            "_id": "3",
            "name": "Vision Vector Frame",
            "description": "Vision Vector Frame - A deep learning display that controls your mood",
            "$vector": [0.1, 0.05, 0.08, 0.3, 0.6],
        },
    ]

res = collection.insert_many(
        documents=json_query,
        options={"ordered": False},
        partial_failures_allowed=True,
    )
print (res)

# Find documents

## Find a document

In [ ]:
document = collection.find_one(filter={"_id":"4"})
print(document)

## Find documents using vector search

When working with vector stores, there are reserved fields $vector and $similarity.  The $ is how these reserved words are used in the system.

In [ ]:
sort = {"$vector": [0.15, 0.1, 0.1, 0.35, 0.55]}
options = {"limit": 100}

document = collection.find(sort=sort, options=options)
print(document)

## Find documents using vector search and projection

Use "projection" to specify which fields you want to retrieve in your documents from the vector store.  The _id field will always be returned unless you indicate not to include it using `{"_id": 0}`

In [ ]:
sort = {"$vector": [0.15, 0.1, 0.1, 0.35, 0.55]}
options = {"limit": 100}
projection = {"$vector": 1, "$similarity": 1}

document = collection.find(sort=sort, options=options, projection=projection)
print(document)




## Find one document using vector search and projection

In [ ]:
sort = {"$vector": [0.15, 0.1, 0.1, 0.35, 0.55]}
projection = {"$vector": 1}

document = collection.find_one(sort=sort, options={}, projection=projection)
print(document)
    

## Find one and update with vector search

In [ ]:
sort = {"$vector": [0.15, 0.1, 0.1, 0.35, 0.55]}
update = {"$set": {"status": "active"}}

collection.find_one_and_update(sort=sort, update=update, options={})
document = collection.find_one(filter={"status": "active"})
print(document)
 

## Find one and replace with vector search

In [ ]:
sort = {"$vector": [0.15, 0.1, 0.1, 0.35, 0.55]}
replacement = {
        "_id": "3",
        "name": "Vision Vector Frame",
        "description": "Vision Vector Frame - A deep learning display that controls your mood",
        "$vector": [0.1, 0.05, 0.08, 0.3, 0.6],
        "status": "inactive",
}

collection.find_one_and_replace(sort=sort, replacement=replacement, options={})
document = collection.find_one(filter={"name": "Vision Vector Frame"})
print(document)
    

## Delete a collection

In [ ]:
returnval = astra_db.delete_collection(collection_name="test_collection")
print(returnval)